# Oxford Data Connector

This file contains the base to connect to the Oxford database and drops the data into a dataframe df. 

## Training

In [103]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta

### Copy the data locally

In [2]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_FILE = 'data/OxCGRT_latest.csv'
#Demographics File
DEMOGRAPHICS_FILE = 'data/Country_Demographics.csv'
DEMOGRAPHICS_BRZ_R_FILE = 'data/brz_region_demo.csv'

In [3]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('data/OxCGRT_latest.csv', <http.client.HTTPMessage at 0x7fbafaa39d60>)

In [31]:
# Load historical data from local file
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

#Data from Worldometers - https://www.worldometers.info/world-population/population-by-country/
demographics = pd.read_csv(DEMOGRAPHICS_FILE,
                 sep = ';',
                 encoding="ISO-8859-1",
                 dtype={"Country": str,
                        "Population": int},
                 error_bad_lines=False)

brz_rg_demo = pd.read_csv(DEMOGRAPHICS_BRZ_R_FILE,
                 sep = ';',
                 encoding="ISO-8859-1",
                 dtype={"Country": str,
                        "Population": int},
                 error_bad_lines=False)

In [6]:
df.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'H7_Vaccination policy', 'H7_Flag',
       'M1_Wildcard', 'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex',
       'StringencyIndexForDisplay', 'StringencyLegacyIndex',
       'StringencyLegacyIndexForDispla

In [7]:
demographics.columns

Index(['Country', 'Population'], dtype='object')

In [8]:
display(df)

,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,...,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
0,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-01,0.0,NaN,0.0,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
1,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-02,0.0,NaN,0.0,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
2,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-03,0.0,NaN,0.0,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
3,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-04,0.0,NaN,0.0,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
4,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-05,0.0,NaN,0.0,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99675,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,2020-12-17,2.0,0.0,1.0,1.0,...,69.44,69.44,73.81,73.81,55.56,55.56,60.26,60.26,25.0,25.0
99676,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,2020-12-18,2.0,0.0,2.0,0.0,...,71.30,71.30,73.81,73.81,56.67,56.67,61.54,61.54,25.0,25.0
99677,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,2020-12-19,2.0,0.0,2.0,0.0,...,71.30,71.30,73.81,73.81,56.67,56.67,61.54,61.54,25.0,25.0
99678,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,2020-12-20,2.0,0.0,2.0,0.0,...,71.30,71.30,73.81,73.81,56.67,56.67,61.54,61.54,25.0,25.0


In [9]:
display(df.set_index('CountryName').filter(like='Brazil',axis=0))

,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,...,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
CountryName,,,,,,,,,,,,,,,,,,,,,
Brazil,BRA,Acre,BR_AC,STATE_TOTAL,2020-01-01,0.0,NaN,0.0,NaN,0.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,NaN,NaN
Brazil,BRA,Acre,BR_AC,STATE_TOTAL,2020-01-02,0.0,NaN,0.0,NaN,0.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,NaN,NaN
Brazil,BRA,Acre,BR_AC,STATE_TOTAL,2020-01-03,0.0,NaN,0.0,NaN,0.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,NaN,NaN
Brazil,BRA,Acre,BR_AC,STATE_TOTAL,2020-01-04,0.0,NaN,0.0,NaN,0.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,NaN,NaN
Brazil,BRA,Acre,BR_AC,STATE_TOTAL,2020-01-05,0.0,NaN,0.0,NaN,0.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Brazil,BRA,NaN,NaN,NAT_TOTAL,2020-12-17,NaN,NaN,NaN,NaN,NaN,...,NaN,59.72,NaN,57.86,NaN,60.28,NaN,61.86,NaN,50.0
Brazil,BRA,NaN,NaN,NAT_TOTAL,2020-12-18,NaN,NaN,NaN,NaN,NaN,...,NaN,59.72,NaN,57.86,NaN,60.28,NaN,61.86,NaN,50.0
Brazil,BRA,NaN,NaN,NAT_TOTAL,2020-12-19,NaN,NaN,NaN,NaN,NaN,...,NaN,59.72,NaN,57.86,NaN,60.28,NaN,61.86,NaN,50.0


In [12]:
#DataFrame.join(other, on=None, how='left', lsuffix='', rsuffix='', sort=False)[source]
test = demographics.join(df.set_index('CountryName'),on='Country',how='inner',lsuffix='Country',rsuffix='CountryName')
display(test.drop_duplicates())

,Country,Population,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_School closing,C1_Flag,C2_Workplace closing,...,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
0,China,1439323776,CHN,NaN,NaN,NAT_TOTAL,2020-01-01,0.0,NaN,0.0,...,0.00,0.00,0.00,0.00,6.39,6.39,7.37,7.37,0.0,0.0
0,China,1439323776,CHN,NaN,NaN,NAT_TOTAL,2020-01-02,0.0,NaN,0.0,...,0.00,0.00,0.00,0.00,6.39,6.39,7.37,7.37,0.0,0.0
0,China,1439323776,CHN,NaN,NaN,NAT_TOTAL,2020-01-03,0.0,NaN,0.0,...,0.00,0.00,0.00,0.00,6.39,6.39,7.37,7.37,0.0,0.0
0,China,1439323776,CHN,NaN,NaN,NAT_TOTAL,2020-01-04,0.0,NaN,0.0,...,0.00,0.00,0.00,0.00,6.39,6.39,7.37,7.37,0.0,0.0
0,China,1439323776,CHN,NaN,NaN,NAT_TOTAL,2020-01-05,0.0,NaN,0.0,...,2.78,2.78,4.76,4.76,11.39,11.39,13.14,13.14,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,San Marino,33931,SMR,NaN,NaN,NAT_TOTAL,2020-12-17,2.0,1.0,1.0,...,44.44,44.44,53.57,53.57,55.00,55.00,51.92,51.92,75.0,75.0
217,San Marino,33931,SMR,NaN,NaN,NAT_TOTAL,2020-12-18,2.0,1.0,1.0,...,51.85,51.85,64.29,64.29,59.44,59.44,57.05,57.05,75.0,75.0
217,San Marino,33931,SMR,NaN,NaN,NAT_TOTAL,2020-12-19,2.0,1.0,1.0,...,51.85,51.85,64.29,64.29,59.44,59.44,57.05,57.05,75.0,75.0
217,San Marino,33931,SMR,NaN,NaN,NAT_TOTAL,2020-12-20,2.0,1.0,1.0,...,51.85,51.85,64.29,64.29,59.44,59.44,57.05,57.05,75.0,75.0


Listing the number of cases

In [34]:
CASES_COLUMNS = ["Country", "Population", "RegionName", "Date", "ConfirmedCases", "ConfirmedDeaths"]
CASES_COLUMNS_REGION = ["Country", "Population", "RegionName","PopulationRegion", "Date", "ConfirmedCases", "ConfirmedDeaths"]

In [14]:
data = test[CASES_COLUMNS]

In [15]:
display(data)

,Country,Population,RegionName,Date,ConfirmedCases,ConfirmedDeaths
0,China,1439323776,NaN,2020-01-01,NaN,NaN
0,China,1439323776,NaN,2020-01-02,NaN,NaN
0,China,1439323776,NaN,2020-01-03,NaN,NaN
0,China,1439323776,NaN,2020-01-04,NaN,NaN
0,China,1439323776,NaN,2020-01-05,NaN,NaN
...,...,...,...,...,...,...
217,San Marino,33931,NaN,2020-12-17,2049.0,54.0
217,San Marino,33931,NaN,2020-12-18,2072.0,55.0
217,San Marino,33931,NaN,2020-12-19,2107.0,55.0
217,San Marino,33931,NaN,2020-12-20,NaN,NaN


In [40]:
data_w_region = test.join(brz_rg_demo.set_index('BrazilRegion'),on='RegionName',how='inner',lsuffix='Country',rsuffix='CountryName')

In [41]:
data_w_region.columns

Index(['Country', 'Population', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'H7_Vaccination policy', 'H7_Flag',
       'M1_Wildcard', 'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex',
       'StringencyIndexForDisplay', 'StringencyLegacyIndex',
       'StringencyLegacyInde

In [43]:
br_data = data_w_region.query('Country == "Brazil" \
                         & ConfirmedCases!=""\
                         & ConfirmedCases!="0"')

In [46]:
br_data = br_data[CASES_COLUMNS_REGION]

In [52]:
br_sp_data = br_data.query('RegionName=="Sao Paulo"\
                            & ConfirmedCases!="NaN"\
                            & ConfirmedCases !="0"') 
display(br_sp_data)

,Country,Population,RegionName,PopulationRegion,Date,ConfirmedCases,ConfirmedDeaths
5,Brazil,212559417,Sao Paulo,46289333,2020-02-26,1.0,0.0
5,Brazil,212559417,Sao Paulo,46289333,2020-02-27,1.0,0.0
5,Brazil,212559417,Sao Paulo,46289333,2020-02-28,1.0,0.0
5,Brazil,212559417,Sao Paulo,46289333,2020-02-29,2.0,0.0
5,Brazil,212559417,Sao Paulo,46289333,2020-03-01,2.0,0.0
...,...,...,...,...,...,...,...
5,Brazil,212559417,Sao Paulo,46289333,2020-12-16,1341428.0,44282.0
5,Brazil,212559417,Sao Paulo,46289333,2020-12-17,1361731.0,44681.0
5,Brazil,212559417,Sao Paulo,46289333,2020-12-18,1371653.0,44878.0
5,Brazil,212559417,Sao Paulo,46289333,2020-12-19,1384100.0,45029.0


In [91]:
subtract = br_sp_data['PopulationRegion']-br_sp_data['ConfirmedCases']
br_sp_data.assign(Susceptible = subtract)

susceptible_percentage = br_sp_data['Susceptible']/br_sp_data['PopulationRegion']
br_sp_data.assign(s = susceptible_percentage)

,Country,Population,RegionName,PopulationRegion,Date,ConfirmedCases,ConfirmedDeaths,Susceptible,s
5,Brazil,212559417,Sao Paulo,46289333,2020-02-26,1.0,0.0,46289332.0,1.000000
5,Brazil,212559417,Sao Paulo,46289333,2020-02-27,1.0,0.0,46289332.0,1.000000
5,Brazil,212559417,Sao Paulo,46289333,2020-02-28,1.0,0.0,46289332.0,1.000000
5,Brazil,212559417,Sao Paulo,46289333,2020-02-29,2.0,0.0,46289331.0,1.000000
5,Brazil,212559417,Sao Paulo,46289333,2020-03-01,2.0,0.0,46289331.0,1.000000
...,...,...,...,...,...,...,...,...,...
5,Brazil,212559417,Sao Paulo,46289333,2020-12-16,1341428.0,44282.0,44947905.0,0.971021
5,Brazil,212559417,Sao Paulo,46289333,2020-12-17,1361731.0,44681.0,44927602.0,0.970582
5,Brazil,212559417,Sao Paulo,46289333,2020-12-18,1371653.0,44878.0,44917680.0,0.970368
5,Brazil,212559417,Sao Paulo,46289333,2020-12-19,1384100.0,45029.0,44905233.0,0.970099


### s = S/N , i = I/N, r = R/N

### s+i+r = 1 

### I+R = ConfirmedCases + ConfirmedDeaths




In [126]:
a = br_sp_data.query(f'Date=="2020-12-20"')

In [166]:
i = 0

sp_I_change = pd.DataFrame({'0' : [0]})
print(sp_I_change)


for d in br_sp_data['Date']:
    if i != 0:
        a = br_sp_data.query(f'Date=="{d}"')
        b = br_sp_data.query(f'Date=="{d-timedelta(days=1)}"')
        c = a["ConfirmedCases"]-b["ConfirmedCases"]
        if c.isnull:
            c = 0
        else:
            c = c.astype(int)
        sp_I_change[i] = c
        i += 1
    else:
        i += 1

print(sp_I_change.transpose())

   0
0  0
     0
0    0
1    0
2    0
3    0
4    0
..  ..
256  0
257  0
258  0
259  0
260  0

[261 rows x 1 columns]
